In [7]:
!pip install -U pip
!pip install transformers
!pip install torch

     -------------------------------------- 162.6/162.6 MB 2.3 MB/s eta 0:00:00


In [8]:
import pandas as pd
import requests
import re
import openai

from collections import Counter
from typing import List, Dict

# from transformers import BertModel, BertTokenizerFast
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

from consts import *

In [ ]:
df = pd.read_csv('kns_csv_files/kns_committee.csv')
df = df[df['KnessetNum'] >= 25]
df = df[df['CategoryID'].isin([MONEY_COM_CATEGORY_ID, DEFENSE_COM_CATEGORY_ID, LAW_ORDER_COM_CATEGORY_ID, MESADERET_COM_CATEGORY_ID, KNESSET_COM_CATEGORY_ID])]
commitee_ids = df['CommitteeID'].to_list()

In [ ]:
def get_meeting_protocol_text(text_path):
    # Define the URL to fetch
    base_url = 'https://production.oknesset.org/pipelines/data/committees/meeting_protocols_text/'
    # Send GET request to the URL
    response = requests.get(base_url + text_path)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        response.encoding = 'utf-8'
        # Retrieve the content of the file
        return response.text
    else:
        raise ValueError(f"Failed to retrieve content. Status code: {response.status_code}")


In [ ]:
com_session_df = pd.read_csv('kns_csv_files/kns_committeesession.csv')
com_session_df = com_session_df[com_session_df['CommitteeID'].isin(commitee_ids)]

com_session_df.dropna(subset=['text_parsed_filename'], inplace=True)
text_paths = com_session_df['text_parsed_filename'].to_list()
texts = [get_meeting_protocol_text(path) for path in text_paths]
agg_scores = [rate_aggressiveness(text) for text in texts]



ConnectionError: HTTPSConnectionPool(host='production.oknesset.org', port=443): Max retries exceeded with url: /pipelines/data/committees/meeting_protocols_text/files/2/1/2197382.txt (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000002624DCDE6D0>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))

In [ ]:
knesset_members_df = pd.read_csv('kns_csv_files/kns_person.csv')
first_names, last_names = knesset_members_df['FirstName'].to_list(), knesset_members_df['LastName'].to_list()
knesset_members = [' '.join([first_name, last_name]) for first_name, last_name in zip(first_names, last_names)]

warnings = {mem: [0, 0, 0] for mem in knesset_members}

# handle members with a middle name or a nickname
new_first_names, new_last_names = [], []

for fn, ln in zip(first_names, last_names):
    names = re.findall('\w+', fn)
    
    for name in names:
        warnings[name + ' ' + ln] = warnings[fn + ' ' + ln]
        new_first_names.append(name)
        new_last_names.append(ln)

# update first and last names
first_names = new_first_names
last_names = new_last_names

knesset_members = [' '.join([first_name, last_name]) for first_name, last_name in zip(first_names, last_names)]

In [ ]:
def get_meeting_warnings(text, warnings, knesset_members) -> None:
    """
    Return warnings from the meeting protocol text.

    Parameters
    ----------
    text : str
        Meeting protocol text.

    warnings: Dict[str, List[int]]
        Number of warnings for each Knesset member.

    knesset_members: List[str]
        List of Knesset members.
    """

    # find all warnings
    matches = re.findall(WARNING_REGEX, text, flags=re.MULTILINE)
    print(len(matches))
    for i, match in enumerate(matches):
        print(f'match #{i}:')
        print(match)
        sentences = match.split('\n')
        first_sentence, last_sentence = sentences[0], sentences[-1]
        for kns_member in knesset_members:
            if kns_member in first_sentence:
                word2idx = {'ראש': 0, 'שני': 1, 'שליש': 2}
                for word, idx in word2idx.items():
                    if word in last_sentence:
                        warnings[kns_member][idx] += 1
                        break
    
    

In [ ]:
alephbert_tokenizer = AutoTokenizer.from_pretrained('onlplab/alephbert-base')
alephbert = AutoModelForSequenceClassification.from_pretrained('onlplab/alephbert-base')

Some weights of the model checkpoint at onlplab/alephbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at onlplab/alephbert-base

In [ ]:
# if not finetuning - disable dropout
alephbert.eval()
text = 'מי אתה בכלל שתרים את הקול שלך עליי?'

inputs = alephbert_tokenizer(text, return_tensors='pt')
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

with torch.no_grad():
    outputs = alephbert(input_ids, attention_mask=attention_mask)
    print('output:', outputs)

    logits = outputs.logits
    print('logists shape:', logits.shape)
    print('logits:', logits)
    # pred = torch.argmax(logits, dim=1)
    # print(pred.item())

output: SequenceClassifierOutput(loss=None, logits=tensor([[0.3183, 0.0073]]), hidden_states=None, attentions=None)
logists shape: torch.Size([1, 2])
logits: tensor([[0.3183, 0.0073]])


In [13]:
def prompt_chatgpt(prompt: str) -> str:
    openai.api_key = None
    response = openai.ChatCompletion.create(
        engine="gpt-3.5-turbo",
        messages=[
            {'role': 'system', 'content': 'You are a chatbot'},
            {'role': 'user', 'content': prompt}
        ]
    )

    return response.choices[0].text

In [14]:
def rate_aggressiveness(text):
    ind = re.search("<< יור >>", text)
    txt2 = text[ind.span()[0]:]
    txt2 = re.sub("<<.*","", txt2)
    txt2 = re.sub(">>.*","", txt2)
    txt2 = re.sub("\n\s+","\n", txt2)
    txt2 = re.sub(" +"," ", txt2)
    lines = txt2.split('\n')

    # Filter non-conversation related lines
    lines = [line for line in lines if line.strip() != '' and '>' not in line and '<' not in line]

    # Merge lines
    text = '\n'.join(lines)

    # Prompt Chat-GPT to rate the aggressiveness of the text
    prompt = 'דרג את מידת האגרסיביות (תשובה מספרית בלבד! מ-1 עד 5 כש-5 זה אגרסיבי מאוד)'
    prompt = prompt + '\n' + text

    response = prompt_chatgpt(prompt)
    return int(response)
    

In [15]:
text = '''

הכנסת העשרים-וארבע

הכנסת



2
ועדת הכנסת
25/04/2022


03/05/2022
10:35
מושב שני



פרוטוקול מס' 89
מישיבת ועדת הכנסת
יום שני, כ"ד בניסן התשפ"ב (25 באפריל 2022), שעה 10:00



סדר היום:
 << נושא >> פניית ראש הממשלה ויו"ר ימינה, חה"כ נפתלי בנט, בבקשה להכריז על חבר הכנסת עמיחי שיקלי כפורש מסיעת ימינה. << נושא >>   


נכחו:
חברי הוועדה: 
איתן גינזבורג – מ"מ היו"ר
רם שפע – מ"מ היו"ר
בועז טופורובסקי
אחמד טיבי
יבגני סובה
יצחק פינדרוס


חברי הכנסת:
יולי אדלשטיין
אמיר אוחנה
ינון אזולאי
דוד אמסלם
אופיר אקוניס
משה ארבל
בני בגין
ולדימיר בליאק
איתמר בן גביר
יואב בן צור
גילה גמליאל
צחי הנגבי
מיכל וולדיגר
משה טור פז
אלי כהן
אופיר כץ
אורלי לוי אבקסיס
יריב לוין
אבי מעוז
אורי מקלב
יעקב מרגי
אופיר סופר
עידית סילמן
אורית סטרוק
שירלי פינטו
יואב קיש
שלמה קרעי
מירי רגב
מיכל רוזין
קטי שטרית
עמיחי שיקלי
נירה שפק


מוזמנים: 
שר הדתות מתן כהנא
מזכיר הכנסת דן מרזוק
עו"ד גיא בוסי 		– מייצג את חבר הכנסת עמיחי שיקלי
עו"ד עדי סדינסקי-לוי 	– יועץ משפטי לסיעת ימינה 
עו"ד עמיחי ויינברג 	– יועץ משפטי לסיעת ימינה


ייעוץ משפטי: 
ארבל אסטרחן

מנהלת הוועדה:
נועה בירן-דדון

רישום פרלמנטרי:
שלומית יוסף;
הילה לוי;
שרון רפאלי;
חנה כהן;
יפה קרינצה;
יפעת קדם;
מאיר פרץ;
אור שושני;
טלי רם;
הילה מליחי;
ירון קוונשטוק



 << נושא >> פניית ראש הממשלה ויו"ר ימינה, חה"כ נפתלי בנט, בבקשה להכריז על חבר הכנסת עמיחי שיקלי כפורש מסיעת ימינה. << נושא >>   

 << יור >> היו"ר איתן גינזבורג: << יור >>   

בוקר טוב לכולם. אני מבקש לאפשר לחברי הכנסת מקום לשבת מסביב לשולחן. 

 << דובר >> דוד אמסלם (הליכוד): << דובר >>   

הייתם צריכים לעשות במקום יותר גדול. אם הייתה לכם כוונה טובה ולא לשואו, אז בואו תעשו את זה. לא הבנתי, אתה קובע. 

 << יור >> היו"ר איתן גינזבורג: << יור >>   

נכון. אז תן לי עכשיו לנהל את הדיון. 

 << דובר >> דוד אמסלם (הליכוד): << דובר >>   

גם היום אתם גונבים, כשאתם 60, וכבר התפרקתם. 

 << יור >> היו"ר איתן גינזבורג: << יור >>   

עוד לא התחלנו, דודי. תן לי רגע. 

 << דובר >> דוד אמסלם (הליכוד): << דובר >>   

עוד לא התחלנו ותראה מה אתם עושים. מה יקרה כשתתחילו? 

 << יור >> היו"ר איתן גינזבורג: << יור >>   

אני סומך עליך. בבקשה, אני רואה פה חברי כנסת שעומדים, אז תאפשרו להם מקום, ובטח לחברי הוועדה שצריכים להיות נוכחים לאורך כל הדיון. 

 << דובר >> דוד אמסלם (הליכוד): << דובר >>   

אתם הרי מתביישים במה שאתם עושים. הכנסתם אזרחים עוד לפני שהגענו. 

 << יור >> היו"ר איתן גינזבורג: << יור >>   

יש פה גם הרבה מאוד יועצים. אני מציע שיועצים – יש פה לשכות שלמות. יועץ אחד לכל חבר כנסת נראה לי מספיק, ואם יהיה צורך גם נוציא יועצים. אז תאפשרו למי שבאמת נדרש להיות פה בדיון להיות בדיון כדי שנוכל לנהל אותו. 

 << דובר >> עמיחי שיקלי (ימינה): << דובר >>   

חבר הכנסת גינזבורג, אני  מבקש שיתאפשר למועמדים של הרשימה להיכנס. כי הוציאו אנשים לפני תחילת הדיון, וזה לא מקובל. הם צריכים להיות פה. 

 << יור >> היו"ר איתן גינזבורג: << יור >>   

בוקר טוב לכולם. אני מתכבד לפתוח את ישיבת ועדת הכנסת. ראשית אני רוצה להביע את תנחומיי ולהשתתף בצערו של יושב-ראש ועדת הכנסת, חבר הכנסת ניר אורבך, שאמו יעל נפטרה השבוע, וגם חמותו זלדה שנפטרה גם היא השבוע. אני שולח מכאן חיבוק חזק וחיזוק למשפחת אורבך כולה שעוברת שבוע כואב במיוחד. יהי זכרן ברוך. 

ניר אורבך ביקש ממני למלא את מקומו בניהול הדיון היום, ולכן אני אנהל את הדיון. על סדר היום פניית יושב-ראש ימינה, ראש הממשלה נפתלי בנט, להכריז על חבר הכנסת עמיחי שיקלי כעל פורש מסיעת ימינה לפי סעיף 6א לחוק יסוד: הכנסת. 

חברים, אנחנו לא נמצאים בדיון רגיל. זה דיון לא רגיל שלא קורה הרבה בכנסת. זה דיון אישי ויש לו אופי שונה משאר הדיונים בוועדה. 

 << קריאה >> קריאה: << קריאה >>   

אופי נכלולי. 

 << יור >> היו"ר איתן גינזבורג: << יור >>   

יש עלינו, חברי הוועדה, אחריות גדולה להיכנס ולהקשיב לדברים. 

 << דובר >> אמיר אוחנה (הליכוד): << דובר >>   

סליחה, יש פה עיתונאים בחוץ. יש פה עורכת הדין כנרת בראשי. אני מבקש להכניס. 

 << יור >> היו"ר איתן גינזבורג: << יור >>   

אני מבקש לא להפריע לי כשאני מדבר בדברי הפתיחה שלי, וזה נראה לי הדבר הכי מינימאלי שאפשר לבקש. 

 << דובר >> אמיר אוחנה (הליכוד): << דובר >>   

לא, הדבר הכי מינימלי שהדיון הזה יהיה פומבי, פתוח, כפי שצריך להיות. 

 << יור >> היו"ר איתן גינזבורג: << יור >>   

עוד לא התחיל הדיון, וכבר מפריעים ליושב-ראש לדבר. 

 << דובר >> דוד אמסלם (הליכוד): << דובר >>   

קודם כל תתנהל בכלל בפתיחה כמו בן-אדם. טוב שלא השארת אותנו בחוץ ואמרת שלא נותנים לך לדבר.
'''

rate_aggressiveness(text)

InvalidRequestError: Invalid URL (POST /v1/engines/gpt-3.5-turbo/chat/completions)

In [ ]:
dir(output)

['__annotations__',
 '__class__',
 '__contains__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__post_init__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'attentions',
 'clear',
 'copy',
 'cross_attentions',
 'fromkeys',
 'get',
 'hidden_states',
 'items',
 'keys',
 'last_hidden_state',
 'move_to_end',
 'past_key_values',
 'pooler_output',
 'pop',
 'popitem',
 'setdefault',
 'to_tuple',
 'update',
 'values']

In [ ]:
dir(output[0])

['H',
 'T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__complex__',
 '__contains__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__dlpack__',
 '__dlpack_device__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__idiv__',
 '__ifloordiv__',
 '__ilshift__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed_